<a href="https://colab.research.google.com/github/orellanomariana/repo_colppy/blob/main/colppy_baja_diario_v02_para_google_drive_credenciales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import requests
import hashlib
import json
import pandas as pd
!pip install --upgrade gspread pandas gspread_dataframe


In [79]:
import gspread
from google.oauth2.service_account import Credentials

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

creds = Credentials.from_service_account_file("credentials.json", scopes=SCOPES)
gc = gspread.authorize(creds)


In [80]:


# URL de la API
URL = "https://login.colppy.com/lib/frontera2/service.php"

# Credenciales de autenticación (usuario de desarrollador en MD5)
auth_payload = {
    "auth": {
        "usuario": "orellano.mariana@gmail.com",  # Usuario de desarrollador
        "password": "a4a0988e0b0c94919be83a6effd1a8bd"  # Contraseña en formato MD5
    },
    "service": {
        "provision": "Usuario",
        "operacion": "iniciar_sesion"
    },
    "parameters": {
        "usuario": "Atacyc@atacyc.com.ar",  # Usuario de Colppy
        "password": "bff7653336974f35a40aa9e06fbab150"  # Contraseña en formato MD5
    }
}

# Solicitar clave de sesión
response = requests.post(URL, json=auth_payload, headers={"Content-Type": "application/json"})
session_data = response.json()

In [81]:
# Verificar si la autenticación fue exitosa
if session_data["response"]["success"]:
    clave_sesion = session_data["response"]["data"]["claveSesion"]
    print(f"Clave de sesión obtenida: {clave_sesion}")


Clave de sesión obtenida: c4eaf415fb8a35c0684bbccda29d3a05


In [82]:
clave_sesion

'c4eaf415fb8a35c0684bbccda29d3a05'

In [83]:


# Define los datos de la solicitud
movimientos_payload = {
    "auth": {
        "usuario": "orellano.mariana@gmail.com",  # Usuario API
        "password": "a4a0988e0b0c94919be83a6effd1a8bd"  # Contraseña en MD5
    },
    "service": {
        "provision": "Contabilidad",
        "operacion": "listar_movimientosdiario"
    },
    "parameters": {
        "sesion": {
            "usuario": "Atacyc@atacyc.com.ar",  # Usuario de la empresa
            "claveSesion": clave_sesion  # Clave de sesión obtenida previamente
        },
        "idEmpresa": "89021",
        "fromDate": "2025-01-01",
        "toDate": "2025-12-31",
        "start": 0,
        "limit": 1000
    }
}

# Enviar la solicitud
response = requests.post(URL, json=movimientos_payload, headers={"Content-Type": "application/json"})

# Convertir la respuesta a JSON
movimientos_data = response.json()

# Verificar si la solicitud fue exitosa
if movimientos_data.get("response", {}).get("success"):
    print("Movimientos obtenidos correctamente:")
    print(json.dumps(movimientos_data, indent=4, ensure_ascii=False))
else:
    print(f"Error al obtener movimientos: {movimientos_data.get('response', {}).get('message', 'Error desconocido')}")


Movimientos obtenidos correctamente:
{
    "service": {
        "provision": "Contabilidad",
        "operacion": "listar_movimientosdiario",
        "version": "1_0_0_0",
        "response_date": "2025-04-09 14:11:26"
    },
    "result": {
        "estado": 0,
        "mensaje": "La operación se realizó correctamente"
    },
    "response": {
        "success": true,
        "message": "La operación se realizó con éxito.",
        "movimientos": [
            {
                "idEmpresa": "89021",
                "idTabla": "8",
                "idElemento": "18567315",
                "idDiario": "439887262",
                "idElementoContacto": "3",
                "idObjetoContacto": "3705252",
                "FechaContabilizado": "2025-01-14 15:44:27",
                "fechaContable": "2025-01-14",
                "idPlanCuenta": "2.1.002.000.00.0",
                "idSubdiario": "1",
                "DebitoCredito": "C",
                "Importe": "19304.00000",
             

In [84]:

print(json.dumps(movimientos_data, indent=4, ensure_ascii=False))

{
    "service": {
        "provision": "Contabilidad",
        "operacion": "listar_movimientosdiario",
        "version": "1_0_0_0",
        "response_date": "2025-04-09 14:11:26"
    },
    "result": {
        "estado": 0,
        "mensaje": "La operación se realizó correctamente"
    },
    "response": {
        "success": true,
        "message": "La operación se realizó con éxito.",
        "movimientos": [
            {
                "idEmpresa": "89021",
                "idTabla": "8",
                "idElemento": "18567315",
                "idDiario": "439887262",
                "idElementoContacto": "3",
                "idObjetoContacto": "3705252",
                "FechaContabilizado": "2025-01-14 15:44:27",
                "fechaContable": "2025-01-14",
                "idPlanCuenta": "2.1.002.000.00.0",
                "idSubdiario": "1",
                "DebitoCredito": "C",
                "Importe": "19304.00000",
                "idTablaAplicado": null,
         

In [85]:


# Verifica si "response" y "movimientos" existen en la respuesta
if "response" in movimientos_data and "movimientos" in movimientos_data["response"]:
    df = pd.DataFrame(movimientos_data["response"]["movimientos"])
    print(df.head())  # Muestra los primeros registros
else:
    print("No se encontraron movimientos en la respuesta.")


  idEmpresa idTabla idElemento   idDiario idElementoContacto idObjetoContacto  \
0     89021       8   18567315  439887262                  3          3705252   
1     89021       8   18567315  439887263                  3          3705252   
2     89021       8   18567583  439894511                  3          3705279   
3     89021       8   18567583  439894512                  3          3705279   
4     89021       8   18682986  442556843                  3          3699993   

    FechaContabilizado fechaContable      idPlanCuenta idSubdiario  ...  \
0  2025-01-14 15:44:27    2025-01-14  2.1.002.000.00.0           1  ...   
1  2025-01-14 15:44:27    2025-01-14  5.1.012.000.00.0           1  ...   
2  2025-01-14 16:00:48    2025-01-14  2.1.002.000.00.0           1  ...   
3  2025-01-14 16:00:48    2025-01-14  5.1.033.000.00.0           1  ...   
4  2025-01-26 17:51:03    2025-01-26  2.1.002.000.00.0           1  ...   

  idElementoAplicado idItem idItemAplicado ccosto1 ccosto2 Con

In [86]:
df

,idEmpresa,idTabla,idElemento,idDiario,idElementoContacto,idObjetoContacto,FechaContabilizado,fechaContable,idPlanCuenta,idSubdiario,...,idElementoAplicado,idItem,idItemAplicado,ccosto1,ccosto2,Conciliado,batch,idTercero,isNIIF,itemId
0,89021,8,18567315,439887262,3,3705252,2025-01-14 15:44:27,2025-01-14,2.1.002.000.00.0,1,...,None,0,0,,,N,None,15039665,0,None
1,89021,8,18567315,439887263,3,3705252,2025-01-14 15:44:27,2025-01-14,5.1.012.000.00.0,1,...,None,0,0,None,None,N,None,15039665,0,None
2,89021,8,18567583,439894511,3,3705279,2025-01-14 16:00:48,2025-01-14,2.1.002.000.00.0,1,...,None,0,0,,,N,None,15039898,0,None
3,89021,8,18567583,439894512,3,3705279,2025-01-14 16:00:48,2025-01-14,5.1.033.000.00.0,1,...,None,0,0,None,None,N,None,15039898,0,None
4,89021,8,18682986,442556843,3,3699993,2025-01-26 17:51:03,2025-01-26,2.1.002.000.00.0,1,...,None,0,0,,,N,None,15006538,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,89021,24,6983582,452615574,None,None,2025-03-11 16:39:59,2025-02-07,2.2.003.000.00.0,5,...,None,0,0,None,None,N,None,None,0,None
996,89021,24,6983582,452615575,None,None,2025-03-11 16:39:59,2025-02-07,2.2.004.000.00.0,5,...,None,1,0,None,None,N,None,None,0,None
997,89021,24,6983582,452615576,None,None,2025-03-11 16:39:59,2025-02-07,1.2.000.000.00.0,5,...,None,2,0,None,None,N,None,None,0,None
998,89021,24,6983587,452616168,None,None,2025-03-11 16:41:35,2025-02-24,2.1.004.000.00.0,5,...,None,0,0,None,None,N,None,None,0,None


In [87]:
df.shape

(1000, 23)

In [88]:
# Filtrar solo cuentas cuyo idPlanCuenta comienza con "4" o "5"
df_filtrado = df[df["idPlanCuenta"].astype(str).str.startswith(("1", "2", "3", "4", "5"))]

# Mostrar resultado
df_filtrado

,idEmpresa,idTabla,idElemento,idDiario,idElementoContacto,idObjetoContacto,FechaContabilizado,fechaContable,idPlanCuenta,idSubdiario,...,idElementoAplicado,idItem,idItemAplicado,ccosto1,ccosto2,Conciliado,batch,idTercero,isNIIF,itemId
0,89021,8,18567315,439887262,3,3705252,2025-01-14 15:44:27,2025-01-14,2.1.002.000.00.0,1,...,None,0,0,,,N,None,15039665,0,None
1,89021,8,18567315,439887263,3,3705252,2025-01-14 15:44:27,2025-01-14,5.1.012.000.00.0,1,...,None,0,0,None,None,N,None,15039665,0,None
2,89021,8,18567583,439894511,3,3705279,2025-01-14 16:00:48,2025-01-14,2.1.002.000.00.0,1,...,None,0,0,,,N,None,15039898,0,None
3,89021,8,18567583,439894512,3,3705279,2025-01-14 16:00:48,2025-01-14,5.1.033.000.00.0,1,...,None,0,0,None,None,N,None,15039898,0,None
4,89021,8,18682986,442556843,3,3699993,2025-01-26 17:51:03,2025-01-26,2.1.002.000.00.0,1,...,None,0,0,,,N,None,15006538,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,89021,24,6983582,452615574,None,None,2025-03-11 16:39:59,2025-02-07,2.2.003.000.00.0,5,...,None,0,0,None,None,N,None,None,0,None
996,89021,24,6983582,452615575,None,None,2025-03-11 16:39:59,2025-02-07,2.2.004.000.00.0,5,...,None,1,0,None,None,N,None,None,0,None
997,89021,24,6983582,452615576,None,None,2025-03-11 16:39:59,2025-02-07,1.2.000.000.00.0,5,...,None,2,0,None,None,N,None,None,0,None
998,89021,24,6983587,452616168,None,None,2025-03-11 16:41:35,2025-02-24,2.1.004.000.00.0,5,...,None,0,0,None,None,N,None,None,0,None


In [89]:
df_filtrado.columns

Index(['idEmpresa', 'idTabla', 'idElemento', 'idDiario', 'idElementoContacto',
       'idObjetoContacto', 'FechaContabilizado', 'fechaContable',
       'idPlanCuenta', 'idSubdiario', 'DebitoCredito', 'Importe',
       'idTablaAplicado', 'idElementoAplicado', 'idItem', 'idItemAplicado',
       'ccosto1', 'ccosto2', 'Conciliado', 'batch', 'idTercero', 'isNIIF',
       'itemId'],
      dtype='object')

In [90]:
df_filtrado_columnas = df_filtrado[["fechaContable", "idPlanCuenta", "DebitoCredito", "Importe",'idElementoContacto', "idObjetoContacto"]]
df_filtrado_columnas # defino columnas que quiero ver

,fechaContable,idPlanCuenta,DebitoCredito,Importe,idElementoContacto,idObjetoContacto
0,2025-01-14,2.1.002.000.00.0,C,19304.00000,3,3705252
1,2025-01-14,5.1.012.000.00.0,D,19304.00000,3,3705252
2,2025-01-14,2.1.002.000.00.0,C,900.00000,3,3705279
3,2025-01-14,5.1.033.000.00.0,D,900.00000,3,3705279
4,2025-01-26,2.1.002.000.00.0,C,92000.00000,3,3699993
...,...,...,...,...,...,...
995,2025-02-07,2.2.003.000.00.0,D,4153301.06000,None,None
996,2025-02-07,2.2.004.000.00.0,D,60423.60000,None,None
997,2025-02-07,1.2.000.000.00.0,C,4213724.66000,None,None
998,2025-02-24,2.1.004.000.00.0,D,16824.60000,None,None


In [91]:
df_filtrado_columnas['Importe'] = pd.to_numeric(df_filtrado_columnas['Importe'], errors='coerce')
df_filtrado_columnas['Importe'] = df_filtrado_columnas['Importe'].apply(lambda x: int(x) if x == int(x) else x)
df_filtrado_columnas

<ipython-input-91-9adb25c6704e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_columnas['Importe'] = pd.to_numeric(df_filtrado_columnas['Importe'], errors='coerce')
<ipython-input-91-9adb25c6704e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_columnas['Importe'] = df_filtrado_columnas['Importe'].apply(lambda x: int(x) if x == int(x) else x)


,fechaContable,idPlanCuenta,DebitoCredito,Importe,idElementoContacto,idObjetoContacto
0,2025-01-14,2.1.002.000.00.0,C,19304.00,3,3705252
1,2025-01-14,5.1.012.000.00.0,D,19304.00,3,3705252
2,2025-01-14,2.1.002.000.00.0,C,900.00,3,3705279
3,2025-01-14,5.1.033.000.00.0,D,900.00,3,3705279
4,2025-01-26,2.1.002.000.00.0,C,92000.00,3,3699993
...,...,...,...,...,...,...
995,2025-02-07,2.2.003.000.00.0,D,4153301.06,None,None
996,2025-02-07,2.2.004.000.00.0,D,60423.60,None,None
997,2025-02-07,1.2.000.000.00.0,C,4213724.66,None,None
998,2025-02-24,2.1.004.000.00.0,D,16824.60,None,None


In [92]:
# Reemplazar valores NaN o None por 0 en la columna 'idElementoContacto'
df_filtrado_columnas['idElementoContacto'] = df_filtrado_columnas['idElementoContacto'].fillna(0).astype(int)


<ipython-input-92-653d807530b5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_columnas['idElementoContacto'] = df_filtrado_columnas['idElementoContacto'].fillna(0).astype(int)


In [93]:
# convierto a int
df_filtrado_columnas["idElementoContacto"] = df_filtrado_columnas["idElementoContacto"].astype(int)


<ipython-input-93-a30696454dd4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_columnas["idElementoContacto"] = df_filtrado_columnas["idElementoContacto"].astype(int)


In [94]:
# Reemplazar valores NaN o None por 0 en la columna 'idElementoContacto'
df_filtrado_columnas['idObjetoContacto'] = df_filtrado_columnas['idObjetoContacto'].fillna(0).astype(int)

<ipython-input-94-d1ed5f43240a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_columnas['idObjetoContacto'] = df_filtrado_columnas['idObjetoContacto'].fillna(0).astype(int)


In [95]:
df_filtrado_columnas["idObjetoContacto"] = df_filtrado_columnas["idObjetoContacto"].astype(int)

<ipython-input-95-caafa9c11f05>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_columnas["idObjetoContacto"] = df_filtrado_columnas["idObjetoContacto"].astype(int)


In [96]:
id_clientes_unicos = df_filtrado_columnas[df_filtrado_columnas["idElementoContacto"] == 18]["idObjetoContacto"].unique()
id_clientes_unicos  # Verifica si devuelve valores

array([11419457, 11399969, 11400021, 11399990, 11419477, 11400013,
       11400026, 11400037, 11400042, 11400051, 11399975, 11400055,
       11400054, 11419734, 11399978, 11400048, 11400066, 11400031,
       11816869])

In [97]:
id_proveedores_unicos = df_filtrado_columnas[df_filtrado_columnas["idElementoContacto"] == 3]["idObjetoContacto"].unique()
id_proveedores_unicos  # Verifica si devuelve valores

array([3705252, 3705279, 3699993, 3694180, 3694264, 3698859, 3702233,
       3694166, 3694187, 3694273, 3702428, 3694172, 3698861, 3694170,
       3694183, 3716747, 3716748, 3694188, 3698862, 3723611, 3694276,
       3698863, 3724070, 3724075, 3692047, 3694174, 3702356, 3726069,
       3726151, 3730156, 3694281, 3741461, 3741808, 3742066, 3742221,
       3746520, 3746525, 3746530, 3747345, 3746543, 3694269, 3694266,
       3748316, 3752906, 3752913, 3752921, 3752925, 3694198, 3761387,
       3762684, 3694175, 3764287, 3765999, 3766949, 3767727, 3694196,
       3769546, 3770087, 3694184, 3776923, 3778246, 3780035, 3742019,
       3782672, 3694167, 3731729])

In [98]:
# ID del cliente a consultar (ajusta este valor)
id_cliente = 11419734

# Función para obtener los detalles del cliente
def leer_detalle_cliente(id_cliente):
    data = {
        "auth": {
            "usuario": "orellano.mariana@gmail.com",
            "password": "a4a0988e0b0c94919be83a6effd1a8bd"
        },
        "service": {
            "provision": "Cliente",
            "operacion": "leer_detalle_cliente"
        },
        "parameters": {
            "sesion": {
                "usuario": "Atacyc@atacyc.com.ar",
                "claveSesion": clave_sesion  # Usa la clave de sesión válida
            },
            "idEmpresa": "89021",
            "idCliente": str(id_cliente)  # Convertir a string por seguridad
        }
    }

    response = requests.post(URL, json=data, headers={"Content-Type": "application/json"})
    return response.json()

# Ejecutar la petición para el cliente especificado
detalle_cliente = leer_detalle_cliente(id_cliente)

# Mostrar resultado
print(json.dumps(detalle_cliente, indent=4, ensure_ascii=False))


{
    "service": {
        "provision": "Cliente",
        "operacion": "leer_detalle_cliente",
        "version": "1_0_0_0",
        "response_date": "2025-04-09 14:11:27"
    },
    "result": {
        "estado": 0,
        "mensaje": "La operación se realizó correctamente"
    },
    "response": {
        "success": true,
        "message": "La operación se realizó con éxito.",
        "data": {
            "idEmpresa": "89021",
            "idCliente": "11419734",
            "RazonSocial": "PAYWAY SAU",
            "NombreFantasia": "PAYWAY SAU",
            "FechaAlta": "07-01-2025",
            "DirPostal": "LAVARDEN 247",
            "DirPostalCiudad": "",
            "DirPostalCodigoPostal": "1437",
            "DirPostalProvincia": "CABA",
            "DirPostalPais": null,
            "DirFiscal": "",
            "DirFiscalCiudad": "",
            "DirFiscalCodigoPostal": "",
            "DirFiscalProvincia": "CABA",
            "DirFiscalPais": "",
            "Telefono": ""

In [99]:
# ID del proveedor a consultar (ajusta este valor)
id_proveedor = 3752925

# Función para obtener los detalles del cliente
def leer_detalle_proveedor(id_proveedor):
    data = {
        "auth": {
            "usuario": "orellano.mariana@gmail.com",
            "password": "a4a0988e0b0c94919be83a6effd1a8bd"
        },
        "service": {
            "provision": "Proveedor",
            "operacion": "leer_proveedor"
        },
        "parameters": {
            "sesion": {
                "usuario": "Atacyc@atacyc.com.ar",
                "claveSesion": clave_sesion  # Usa la clave de sesión válida
            },
            "idProveedor": str(id_proveedor),  # Convertir a string por seguridad
            "idEmpresa": "89021"

        }
    }

    response = requests.post(URL, json=data, headers={"Content-Type": "application/json"})
    return response.json()

# Ejecutar la petición para el proveedor especificado
detalle_proveedor = leer_detalle_proveedor(id_proveedor)

# Mostrar resultado
print(json.dumps(detalle_proveedor, indent=4, ensure_ascii=False))

{
    "service": {
        "provision": "Proveedor",
        "operacion": "leer_proveedor",
        "version": "1_0_0_0",
        "response_date": "2025-04-09 14:11:27"
    },
    "result": {
        "estado": 0,
        "mensaje": "La operación se realizó correctamente"
    },
    "response": {
        "success": true,
        "message": "La operación se realizó con éxito.",
        "detalle": {
            "idEmpresa": "89021",
            "idProveedor": "3752925",
            "RazonSocial": "GASTON SANDLER",
            "NombreFantasia": "GASTON SANDLER",
            "FechaAlta": "2025-03-07",
            "DirPostal": "COSTA RICA 6050 Piso:05 Dpto:C",
            "DirPostalCiudad": "",
            "DirPostalCodigoPostal": "1414",
            "DirPostalProvincia": "CABA",
            "DirPostalPais": "Argentina",
            "DirFiscal": "",
            "DirFiscalCiudad": "",
            "DirFiscalCodigoPostal": "1414",
            "DirFiscalProvincia": "CABA",
            "DirFiscal

In [100]:

# Lista para almacenar los resultados
clientes_data = []

# Iterar sobre cada id_cliente único
for id_cliente in id_clientes_unicos:
    print(f"Consultando API con ID: {id_cliente}")  # Imprimir el ID que estamos consultando
    detalle_cliente = leer_detalle_cliente(id_cliente)

    # Verificar la respuesta completa de la API
    print(f"Respuesta de la API para ID {id_cliente}:")
    print(json.dumps(detalle_cliente, indent=4, ensure_ascii=False))  # Imprimir respuesta completa

    # Acceder a la RazonSocial dentro de la respuesta
    try:
        razon_social = detalle_cliente.get("response", {}).get("data", {}).get("RazonSocial", None)
    except KeyError:
        razon_social = None

    # Agregar los resultados a la lista como un diccionario
    clientes_data.append({"idObjetoContacto": id_cliente, "RazonSocial": razon_social})

# Convertir la lista de diccionarios a un DataFrame
df_clientes = pd.DataFrame(clientes_data)

Consultando API con ID: 11419457
Respuesta de la API para ID 11419457:
{
    "service": {
        "provision": "Cliente",
        "operacion": "leer_detalle_cliente",
        "version": "1_0_0_0",
        "response_date": "2025-04-09 14:11:28"
    },
    "result": {
        "estado": 0,
        "mensaje": "La operación se realizó correctamente"
    },
    "response": {
        "success": true,
        "message": "La operación se realizó con éxito.",
        "data": {
            "idEmpresa": "89021",
            "idCliente": "11419457",
            "RazonSocial": "FIRST DATA CONO SUR SOCIEDAD DE RESPONSABILIDAD LIMITADA",
            "NombreFantasia": "FIRST DATA CONO SUR SOCIEDAD DE RESPONSABILIDAD LIMITADA",
            "FechaAlta": "07-01-2025",
            "DirPostal": "PERU 143",
            "DirPostalCiudad": "",
            "DirPostalCodigoPostal": "1067",
            "DirPostalProvincia": "CABA",
            "DirPostalPais": null,
            "DirFiscal": "",
            "DirFi

In [101]:
df_clientes

,idObjetoContacto,RazonSocial
0,11419457,FIRST DATA CONO SUR SOCIEDAD DE RESPONSABILIDA...
1,11399969,AMERICAN EXPRESS ARGENTINA S A
2,11400021,GETNET ARGENTINA S. A. U.
3,11399990,CENTRO CARD S.A.
4,11419477,CREDIRED SA
5,11400013,CREDIMAS S A
6,11400026,ITALCRED S A
7,11400037,MERCURIO SA
8,11400042,MONTEMAR COMPA#IA FINANCIERA S A
9,11400051,TARJETAS DEL MAR SA


In [102]:
# Lista para almacenar los resultados
proveedores_data = []

# Iterar sobre cada id_proveedor único
for id_proveedor in id_proveedores_unicos:
    print(f"Consultando API con ID: {id_proveedor}")  # Imprimir el ID que estamos consultando
    detalle_proveedor = leer_detalle_proveedor(id_proveedor)

    # Verificar la respuesta completa de la API
    print(f"Respuesta de la API para ID {id_proveedor}:")
    print(json.dumps(detalle_proveedor, indent=4, ensure_ascii=False))  # Imprimir respuesta completa

    # Acceder a la RazonSocial dentro de la respuesta
    try:
        # Asegurándonos de acceder correctamente a 'detalle' dentro de 'response'
        razon_social = detalle_proveedor.get("response", {}).get("detalle", {}).get("RazonSocial", None)

        # Depurar: Verificar qué estamos extrayendo
        print(f"RazonSocial extraída para ID {id_proveedor}: {razon_social}")
    except KeyError as e:
        print(f"Error al acceder a 'RazonSocial' para ID {id_proveedor}: {e}")
        razon_social = None

    # Agregar los resultados a la lista como un diccionario
    proveedores_data.append({"idObjetoContacto": id_proveedor, "RazonSocial": razon_social})

# Convertir la lista de diccionarios a un DataFrame
df_proveedores = pd.DataFrame(proveedores_data)

# Mostrar el DataFrame resultante
print(df_proveedores)

Consultando API con ID: 3705252
Respuesta de la API para ID 3705252:
{
    "service": {
        "provision": "Proveedor",
        "operacion": "leer_proveedor",
        "version": "1_0_0_0",
        "response_date": "2025-04-09 14:11:33"
    },
    "result": {
        "estado": 0,
        "mensaje": "La operación se realizó correctamente"
    },
    "response": {
        "success": true,
        "message": "La operación se realizó con éxito.",
        "detalle": {
            "idEmpresa": "89021",
            "idProveedor": "3705252",
            "RazonSocial": "DELIVERY HERO E-COMMERCE S. A.",
            "NombreFantasia": "PEDIDO YA",
            "FechaAlta": "2025-01-14",
            "DirPostal": "LIBERTADOR DEL AV. 7208 Piso:20",
            "DirPostalCiudad": "",
            "DirPostalCodigoPostal": "1429",
            "DirPostalProvincia": "CABA",
            "DirPostalPais": "Argentina",
            "DirFiscal": "",
            "DirFiscalCiudad": "",
            "DirFiscalCodigo

In [103]:
df_proveedores

,idObjetoContacto,RazonSocial
0,3705252,DELIVERY HERO E-COMMERCE S. A.
1,3705279,BANCO CREDICOOP COOPERATIVO LTDO
2,3699993,VIATICOS ( C/ TICKET)
3,3694180,DANIELA CLARISA MARTIN
4,3694264,TELEFONICA MOVILES ARGENTINA SOCIEDAD ANONIMA
...,...,...
61,3780035,HONG LIN
62,3742019,CONSULTORIA INTEGRAL S A
63,3782672,LEANDRO HERNAN SPATZ
64,3694167,CESAR BAGALA JULIO


In [104]:
# Convertir las columnas idObjetoContacto a tipo int64 en los tres DataFrames
df_filtrado_columnas['idObjetoContacto'] = df_filtrado_columnas['idObjetoContacto'].astype(int)
df_clientes['idObjetoContacto'] = df_clientes['idObjetoContacto'].astype(int)
df_proveedores['idObjetoContacto'] = df_proveedores['idObjetoContacto'].astype(int)

# Realizar el merge entre df_filtrado_columnas, df_clientes y df_proveedores
df_final = pd.merge(df_filtrado_columnas, df_clientes, on='idObjetoContacto', how='left')
df_final = pd.merge(df_final, df_proveedores, on='idObjetoContacto', how='left')

# Mostrar el DataFrame resultante
df_final

<ipython-input-104-6d89e10fc3d3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_columnas['idObjetoContacto'] = df_filtrado_columnas['idObjetoContacto'].astype(int)


,fechaContable,idPlanCuenta,DebitoCredito,Importe,idElementoContacto,idObjetoContacto,RazonSocial_x,RazonSocial_y
0,2025-01-14,2.1.002.000.00.0,C,19304.00,3,3705252,NaN,DELIVERY HERO E-COMMERCE S. A.
1,2025-01-14,5.1.012.000.00.0,D,19304.00,3,3705252,NaN,DELIVERY HERO E-COMMERCE S. A.
2,2025-01-14,2.1.002.000.00.0,C,900.00,3,3705279,NaN,BANCO CREDICOOP COOPERATIVO LTDO
3,2025-01-14,5.1.033.000.00.0,D,900.00,3,3705279,NaN,BANCO CREDICOOP COOPERATIVO LTDO
4,2025-01-26,2.1.002.000.00.0,C,92000.00,3,3699993,NaN,VIATICOS ( C/ TICKET)
...,...,...,...,...,...,...,...,...
995,2025-02-07,2.2.003.000.00.0,D,4153301.06,0,0,NaN,NaN
996,2025-02-07,2.2.004.000.00.0,D,60423.60,0,0,NaN,NaN
997,2025-02-07,1.2.000.000.00.0,C,4213724.66,0,0,NaN,NaN
998,2025-02-24,2.1.004.000.00.0,D,16824.60,0,0,NaN,NaN


In [105]:
# Combinar las dos columnas 'RazonSocial_x' y 'RazonSocial_y'
df_final['RazonSocial'] = df_final['RazonSocial_x'].fillna(df_final['RazonSocial_y'])

# Eliminar las columnas separadas si ya no las necesitas
df_final = df_final.drop(['RazonSocial_x', 'RazonSocial_y'], axis=1)
df_final = df_final.drop(['idElementoContacto', 'idObjetoContacto'], axis=1)

df_final



,fechaContable,idPlanCuenta,DebitoCredito,Importe,RazonSocial
0,2025-01-14,2.1.002.000.00.0,C,19304.00,DELIVERY HERO E-COMMERCE S. A.
1,2025-01-14,5.1.012.000.00.0,D,19304.00,DELIVERY HERO E-COMMERCE S. A.
2,2025-01-14,2.1.002.000.00.0,C,900.00,BANCO CREDICOOP COOPERATIVO LTDO
3,2025-01-14,5.1.033.000.00.0,D,900.00,BANCO CREDICOOP COOPERATIVO LTDO
4,2025-01-26,2.1.002.000.00.0,C,92000.00,VIATICOS ( C/ TICKET)
...,...,...,...,...,...
995,2025-02-07,2.2.003.000.00.0,D,4153301.06,NaN
996,2025-02-07,2.2.004.000.00.0,D,60423.60,NaN
997,2025-02-07,1.2.000.000.00.0,C,4213724.66,NaN
998,2025-02-24,2.1.004.000.00.0,D,16824.60,NaN


In [106]:
# Contar cuántos valores NaN hay en la columna 'RazonSocial'
nan_count = df_final['RazonSocial'].isna().sum()

print(f"Cantidad de valores NaN en la columna 'RazonSocial': {nan_count}")


Cantidad de valores NaN en la columna 'RazonSocial': 53


In [107]:
# Filtrar las filas que tienen al menos un NaN en alguna columna
df_with_nan = df_final[df_final.isna().any(axis=1)]

# Mostrar las filas con NaN
df_with_nan

,fechaContable,idPlanCuenta,DebitoCredito,Importe,RazonSocial
947,2025-01-31,5.1.002.000.00.0,D,11602611.90,NaN
948,2025-01-31,5.1.003.000.00.0,D,3004611.11,NaN
949,2025-01-31,2.3.000.000.00.0,C,8606817.00,NaN
950,2025-01-31,2.2.005.000.00.0,C,1945776.00,NaN
951,2025-01-31,2.2.004.000.00.0,C,60423.60,NaN
952,2025-01-31,2.2.003.000.00.0,C,3994206.41,NaN
953,2025-01-27,1.2.000.000.00.0,D,7700003.36,NaN
954,2025-01-27,1.2.002.000.00.0,C,7700003.36,NaN
955,2025-01-31,1.2.002.000.00.0,D,8500000.00,NaN
956,2025-01-31,1.2.000.000.00.0,C,8500000.00,NaN




*   5.1.026.000.00.0  Seguros
*   5.1.002.000.00.0  sueldos e indemnizaciones
*   5.1.003.000.00.0  Cargas sociales
*   5.1.005.000.00.0  Seguro de vida
*   5.1.033.000.00.0  Comisiones y gastos bancarios
*   4.1.004.000.00.0  Intereses ganados
*   5.1.017.000.00.0  Licencias Microsoft






In [108]:
#listar plan de cuentas


# Autenticación
usuario_api = "orellano.mariana@gmail.com"
password_api = "a4a0988e0b0c94919be83a6effd1a8bd"
usuario_empresa = "Atacyc@atacyc.com.ar"
clave_sesion_empresa = clave_sesion  # Reemplaza con la clave de sesión válida
id_empresa = "89021"

# Encriptar la contraseña en MD5
password_md5 = hashlib.md5(password_api.encode('utf-8')).hexdigest()

# Crear los datos del cuerpo de la solicitud
data = {
    "auth": {
        "usuario": usuario_api,
        "password": password_api
    },
    "service": {
        "provision": "Contabilidad",
        "operacion": "listar_cuentasdiario"
    },
    "parameters": {
        "sesion": {
            "usuario": usuario_empresa,
            "claveSesion": clave_sesion        },
        "idEmpresa": id_empresa,
        "query": ""
    }
}

# URL de la API
URL = "https://login.colppy.com/lib/frontera2/service.php"

# Enviar la solicitud POST
response = requests.post(URL, json=data)

# Verificar el estado de la solicitud y manejar la respuesta
if response.status_code == 200:
    # Convertir la respuesta a formato JSON
    resultado = response.json()
    # Imprimir el resultado
    print(json.dumps(resultado, indent=4, ensure_ascii=False))
else:
    print(f"Error al hacer la solicitud: {response.status_code}")


{
    "service": {
        "provision": "Contabilidad",
        "operacion": "listar_cuentasdiario",
        "version": "1_0_0_0",
        "response_date": "2025-04-09 14:11:57"
    },
    "result": {
        "estado": 0,
        "mensaje": "La operación se realizó correctamente"
    },
    "response": {
        "success": true,
        "message": "La operación se realizó con éxito.",
        "cuentas": [
            {
                "Id": "22",
                "idPlanCuenta": "1.4.000.000.00.0",
                "Descripcion": "1.4.000.000.00.0 - Adelanto asociados",
                "idTipoCuenta": "3",
                "tipoCuenta": "Créditos por Ventas"
            },
            {
                "Id": "27",
                "idPlanCuenta": "1.4.005.000.00.0",
                "Descripcion": "1.4.005.000.00.0 - Adelantos de sueldos",
                "idTipoCuenta": "4",
                "tipoCuenta": "Otros Créditos"
            },
            {
                "Id": "435576",
        

In [109]:


# Guardar la respuesta de la API en un archivo JSON
with open("respuesta_colppy.json", "w", encoding="utf-8") as json_file:
    json.dump(resultado, json_file, ensure_ascii=False, indent=4)


In [110]:


# Cargar el archivo JSON
with open("respuesta_colppy.json", "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

# Acceder a la lista de cuentas contables desde la clave 'cuentas'
if 'response' in data and 'cuentas' in data['response']:
    cuentas_contables = data['response']['cuentas']

    # Crear el DataFrame con los datos de las cuentas contables
    df_cuentas = pd.DataFrame(cuentas_contables)

    # Mostrar el DataFrame
    print(df_cuentas)
else:
    print("No se encontraron cuentas contables o la estructura de la respuesta es diferente.")


         Id      idPlanCuenta  \
0        22  1.4.000.000.00.0   
1        27  1.4.005.000.00.0   
2    435576       4.1.000.000   
3        68  3.3.000.000.00.0   
4        99  5.1.014.000.00.0   
..      ...               ...   
106       8  1.1.002.000.00.0   
107     102  5.1.018.000.00.0   
108       7  1.1.001.005.00.0   
109      80  4.1.007.000.00.0   
110      98  5.1.013.000.00.0   

                                          Descripcion idTipoCuenta  \
0               1.4.000.000.00.0 - Adelanto asociados            3   
1             1.4.005.000.00.0 - Adelantos de sueldos            4   
2          4.1.000.000 - Ajuste configuración inicial           13   
3               3.3.000.000.00.0 - Ajustes de capital           13   
4                       5.1.014.000.00.0 - Alquileres           18   
..                                                ...          ...   
106           1.1.002.000.00.0 - Tarjetas precargadas            1   
107            5.1.018.000.00.0 - Telefono 

In [111]:
df_cuentas

,Id,idPlanCuenta,Descripcion,idTipoCuenta,tipoCuenta
0,22,1.4.000.000.00.0,1.4.000.000.00.0 - Adelanto asociados,3,Créditos por Ventas
1,27,1.4.005.000.00.0,1.4.005.000.00.0 - Adelantos de sueldos,4,Otros Créditos
2,435576,4.1.000.000,4.1.000.000 - Ajuste configuración inicial,13,Patrimonio Neto
3,68,3.3.000.000.00.0,3.3.000.000.00.0 - Ajustes de capital,13,Patrimonio Neto
4,99,5.1.014.000.00.0,5.1.014.000.00.0 - Alquileres,18,Gastos
...,...,...,...,...,...
106,8,1.1.002.000.00.0,1.1.002.000.00.0 - Tarjetas precargadas,1,Caja y Bancos
107,102,5.1.018.000.00.0,5.1.018.000.00.0 - Telefono e internet,18,Gastos
108,7,1.1.001.005.00.0,1.1.001.005.00.0 - Valores a depositar,1,Caja y Bancos
109,80,4.1.007.000.00.0,4.1.007.000.00.0 - Variacion fondos de inversion,14,Ingresos por Ventas


In [112]:
# Separar la columna 'Descripcion' por el guion y quedarte solo con la segunda parte
df_cuentas['Descripcion'] = df_cuentas['Descripcion'].str.split('-', n=1).str[1].str.strip()

# Ahora puedes realizar el merge
base_de_datos = pd.merge(df_final, df_cuentas, on='idPlanCuenta', how='left')

# Mostrar el DataFrame resultante
base_de_datos


,fechaContable,idPlanCuenta,DebitoCredito,Importe,RazonSocial,Id,Descripcion,idTipoCuenta,tipoCuenta
0,2025-01-14,2.1.002.000.00.0,C,19304.00,DELIVERY HERO E-COMMERCE S. A.,46,cuentas a pagar,8,Deudas Comerciales
1,2025-01-14,5.1.012.000.00.0,D,19304.00,DELIVERY HERO E-COMMERCE S. A.,97,Gastos de representacion,18,Gastos
2,2025-01-14,2.1.002.000.00.0,C,900.00,BANCO CREDICOOP COOPERATIVO LTDO,46,cuentas a pagar,8,Deudas Comerciales
3,2025-01-14,5.1.033.000.00.0,D,900.00,BANCO CREDICOOP COOPERATIVO LTDO,113,Comisiones y gastos bancarios,18,Gastos
4,2025-01-26,2.1.002.000.00.0,C,92000.00,VIATICOS ( C/ TICKET),46,cuentas a pagar,8,Deudas Comerciales
...,...,...,...,...,...,...,...,...,...
995,2025-02-07,2.2.003.000.00.0,D,4153301.06,NaN,53,pasivos sociales a pagar,10,Deudas Sociales
996,2025-02-07,2.2.004.000.00.0,D,60423.60,NaN,54,credito utedyc,10,Deudas Sociales
997,2025-02-07,1.2.000.000.00.0,C,4213724.66,NaN,11,Banco credicoop 130607,1,Caja y Bancos
998,2025-02-24,2.1.004.000.00.0,D,16824.60,NaN,48,seguros a pagar,8,Deudas Comerciales


In [113]:
valores_unicos = base_de_datos['idPlanCuenta'].unique()
valores_unicos

array(['2.1.002.000.00.0', '5.1.012.000.00.0', '5.1.033.000.00.0',
       '5.1.013.000.00.0', '5.1.008.000.00.0', '5.1.018.000.00.0',
       '5.1.016.000.00.0', '5.1.015.000.00.0', '5.1.022.000.00.0',
       '5.1.014.000.00.0', '5.1.025.000.00.0', '5.1.021.000.00.0',
       '5.1.039.000.00.0', '5.1.028.000.00.0', '1.6.002.000.00.0',
       '5.1.004.000.00.0', '5.1.029.000.00.0', '1.4.002.000.00.0',
       '5.1.024.000.00.0', '5.1.011.000.00.0', '5.1.036.000.00.0',
       '5.2.000.000.00.0', '5.3.000.000.00.0', '1.2.000.000.00.0',
       '1.1.002.000.00.0', '1.1.002.000.00.1', '1.3.002.000.00.0',
       '4.1.002.000.00.0', '5.1.002.000.00.0', '5.1.003.000.00.0',
       '2.3.000.000.00.0', '2.2.005.000.00.0', '2.2.004.000.00.0',
       '2.2.003.000.00.0', '1.2.002.000.00.0', '2.2.000.000.00.0',
       '5.1.026.000.00.0', '4.1.004.000.00.0', '5.1.017.000.00.0',
       '1.4.006.000.00.0', '5.1.005.000.00.0', '2.1.004.000.00.0'],
      dtype=object)

In [114]:
from google.colab import auth
from googleapiclient.discovery import build
import gspread
from gspread_dataframe import set_with_dataframe
import pandas as pd

# Autenticarse en Google Drive
#auth.authenticate_user()
#from google.colab import auth
#from google.auth import default

#creds, _ = default()
#gc = gspread.authorize(creds)


In [115]:

# Nombre del archivo y carpeta en Drive
sheet_name = "base_de_datos_real_colppy"
folder_id = "1Qg0FVctkRqsatCAXZLGpgChFjoOeVmF1"

# Conectar con Google Drive
drive_service = build('drive', 'v3', credentials=creds)
query = f"name='{sheet_name}' and mimeType='application/vnd.google-apps.spreadsheet' and '{folder_id}' in parents"
response = drive_service.files().list(q=query).execute()
files = response.get('files', [])

if files:
    # Si el archivo existe, abrirlo
    sheet = gc.open(sheet_name)
    worksheet = sheet.sheet1

    # Borrar datos previos
    worksheet.clear()

    print(f"📄 Archivo encontrado. Sobrescribiendo datos en: {sheet_name}")

else:
    # Si no existe, crearlo en la carpeta específica
    file_metadata = {
        'name': sheet_name,
        'mimeType': 'application/vnd.google-apps.spreadsheet',
        'parents': [folder_id]
    }
    file = drive_service.files().create(body=file_metadata).execute()
    sheet = gc.open_by_key(file['id'])
    worksheet = sheet.sheet1

    print(f"📄 Archivo no encontrado. Creando nuevo archivo: {sheet_name}")

# Escribir los nuevos datos en la hoja
set_with_dataframe(worksheet, base_de_datos)

print(f"✅ Datos actualizados en Google Sheets: https://docs.google.com/spreadsheets/d/{sheet.id}")


📄 Archivo encontrado. Sobrescribiendo datos en: base_de_datos_real_colppy
✅ Datos actualizados en Google Sheets: https://docs.google.com/spreadsheets/d/1sXeo0Jk5TeUBv0tjBCdgXSgY5SipExLjW1xGb2dlrnE
